# Capítulo 1 - Introdução

## Motivação Hipotética: DataSciencester

Parabéns! Você acabou de ser contratado para liderar os esforços de data science
na DataSciencester, a rede social para cientistas de dados.

Apesar de ser para os cientistas de dados, a DataSciencester nunca investiu em
construir sua própria atividade de data science (na verdade, a DataSciencester
nunca investiu em construir seu próprio produto). Esse será seu trabalho! No
decorrer do livro, aprenderemos sobre os conceitos de data science ao resolver
problemas com os quais você se depara no trabalho. Algumas vezes, olharemos
para os dados explicitamente fornecidos pelo usuário, outras vezes olharemos
para os gerados por suas interações com um site e, às vezes, olharemos para os
dados dos experimentos que projetaremos.

E, devido à DataSciencester possuir uma forte mentalidade de “não-foiinventado-aqui”, nós construiremos nossas próprias ferramentas do zero. No
final, você terá um sólido entendimento dos fundamentos de data science. Você
estará pronto para aplicar suas habilidades em sua empresa com uma premissa
menos duvidosa, ou em qualquer outro problema que vier a despertar seu
interesse.

Bem-vindo a bordo e boa sorte! ‘Você pode usar jeans às sextas e o toalete é no
final do corredor à direita.’


## Encontrando Conectores-Chave

É seu primeiro dia de trabalho na DataSciencester e o vice-presidente de Rede
(networking) está cheio de perguntas sobre seus usuários. Até agora, ele não teve
ninguém para perguntar, então ele está muito empolgado em ter você aqui.

Particularmente, ele quer que você identifique quem são os “conectores-chave”
entre os cientistas de dados. Para isso, ele lhe dá uma parte de toda a rede da
DataSciencester. Na vida real, você geralmente não recebe os dados de que
precisa. O Capítulo 9 é voltado para a obtenção de dados.

Com o que se parece essa parte dos dados? Ela consiste em uma lista de
usuários, cada um representado por um ``dict`` que contém um ``id`` (um número) para cada usuário ou usuária e um ``name`` (que por uma das grandes coincidências
cósmicas que rima com o ``id`` do usuário):

In [1]:
users = [
    {'id': 0, 'name': 'Hero'},
    {'id': 1, 'name': 'Dunn'},
    {'id': 2, 'name': 'Sue'},
    {'id': 3, 'name': 'Chi'},
    {'id': 4, 'name': 'Thor'},
    {'id': 5, 'name': 'Clive'},
    {'id': 6, 'name': 'Hicks'},
    {'id': 7, 'name': 'Devin'},
    {'id': 8, 'name': 'Kate'},
    {'id': 9, 'name': 'Klein'}
]

Ele também fornece dados “amigáveis”, representados por uma lista de pares de
IDs:

In [2]:
friendships = [
    (0, 1), (0, 2), (1, 2), 
    (1, 3), (2, 3), (3, 4),
    (4, 5), (5, 6), (5, 7), 
    (6, 8), (7, 8), (8, 9)
]

Por exemplo, a tupla ``(0,1)`` indica que o cientista de dados com a ``id`` 0 (Hero) e o
cientista de dados com a ``id`` 1 (Dunn) são amigos.

Já que representamos nossos usuários como ``dicts``, é fácil de aumentá-los com
dados extras.

Por exemplo, talvez nós queiramos adicionar uma lista de amigos para cada
usuário. Primeiro nós configuramos a propriedade ``friends`` de cada usuário em uma
lista vazia:

In [3]:
for user in users:
    user["friends"] = []

Então, nós povoamos a lista com os dados de ``friendships``:

In [4]:
for i, j in friendships:
    # isso funciona porque users[i] é o usuário cuja id é i
    users[i]["friends"].append(users[j]) # adiciona i como um amigo de j
    users[j]["friends"].append(users[i]) # adiciona j como um amigo de i

Uma vez que o ``dict`` de cada usuário contenha uma lista de amigos, podemos
facilmente perguntar sobre nosso gráfico, como “qual é o número médio de
conexões?”

Primeiro, encontramos um número total de conexões, resumindo os tamanhos de
todas as listas de ``friends``:

In [5]:
def number_of_friends(user):
    """quantos amigos o usuário tem?"""
    return len(user["friends"]) # tamanho da lista friend_ids

total_connections = sum(number_of_friends(user) for user in users)

Então, apenas dividimos pelo número de usuários:

In [6]:
num_users = len(users) # tamanho da lista de usuários
avg_connections = total_connections // num_users

Também é fácil de encontrar as pessoas mais conectadas — são as que possuem
o maior número de amigos.

Como não há muitos usuários, podemos ordená-los de “muito amigos” para
“menos amigos”:

In [7]:
# cria uma lista (user_id, number_of_friends)
num_friends_by_id = [(user["id"], number_of_friends(user)) for user in users]

# num_friends_by_id é ordenado por num_friends do maior para o menor
num_friends_by_id.sort(key=lambda x: x[1], reverse=True)

# cada par é (user_id, num_friends)
# [(1, 3), (2, 3), (3, 3), (5, 3), (8, 3),
# (0, 2), (4, 2), (6, 2), (7, 2), (9, 1)]

Uma maneira de pensar sobre o que nós fizemos é uma maneira de identificar as
pessoas que são, de alguma forma, centrais para a rede. Na verdade, o que
acabamos de computar é uma rede métrica de grau de centralidade.



## Cientistas de Dados Que Você Talvez Conheça

Enquanto você está preenchendo os papéis de admissão, a vice-presidente da
Fraternidade chega a sua mesa. Ela quer estimular mais conexões entre os seus
membros, e pede que você desenvolva sugestões de “Cientistas de Dados Que
Você Talvez Conheça”.

Seu primeiro instinto é sugerir um usuário que possa conhecer amigos de
amigos. São fáceis de computar: para cada amigo de um usuário, itera sobre os
amigos daquela pessoa, e coleta todos os resultados:

In [8]:
def friends_of_friend_ids_bad(user):
    # “foaf” é abreviação de “friend of a friend”
    # para cada amigo de usuário pega cada _their_friends
    return [foaf["id"] 
        for friend in user["friends"] 
        for foaf in friend["friends"]
    ]

Quando chamamos ``users[0]`` (Hero), ele produz:

In [9]:
friends_of_friend_ids_bad(users[0])

[0, 2, 3, 0, 1, 3]

Isso inclui o usuário 0 (duas vezes), uma vez que Hero é, de fato, amigo de
ambos os seus amigos. Inclui os usuários 1 e 2, apesar de eles já serem amigos
do Hero. E inclui o usuário 3 duas vezes, já que Chi é alcançável por meio de
dois amigos diferentes:

In [10]:
print ([friend["id"] for friend in users[0]["friends"]])
print ([friend["id"] for friend in users[1]["friends"]]) 
print ([friend["id"] for friend in users[2]["friends"]]) 

[1, 2]
[0, 2, 3]
[0, 1, 3]


Saber que as pessoas são amigas-de-amigas de diversas maneiras parece uma
informação interessante, então talvez nós devêssemos produzir uma contagem de
amigos em comum. Definitivamente, devemos usar uma função de ajuda para
excluir as pessoas que já são conhecidas do usuário:

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4ab8da8e-f281-41bf-9d60-b691c377a948' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>